In [1]:
from google.colab import drive  
drive._mount('/content/drive')  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# importing important libraries
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from tpot import TPOTClassifier

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
x = pd.read_csv("/content/drive/MyDrive/Covid Prediction AI Project/X.csv")
y = pd.read_csv("/content/drive/MyDrive/Covid Prediction AI Project/Y.csv")

In [4]:
x

,Unnamed: 0,cough,fever,sore_throat,shortness_of_breath,head_ache,Female,Male,Age above 60,Abroad,Contact with confirmed
0,0,0,0,0,0,0,1,0,0,0,0
1,1,1,0,0,0,0,1,0,0,0,0
2,2,0,1,0,0,0,0,1,0,0,0
3,3,1,0,0,0,0,1,0,0,0,0
4,4,1,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
520011,520011,1,0,0,0,0,0,1,0,0,1
520012,520012,1,0,0,0,1,1,0,0,0,0
520013,520013,0,0,0,0,0,0,1,0,0,0
520014,520014,0,1,0,0,0,0,1,0,0,0


In [5]:
y

,Unnamed: 0,Result
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
520011,520011,1
520012,520012,1
520013,520013,1
520014,520014,1


In [6]:
x.drop(columns=['Unnamed: 0'], inplace=True)
y.drop(columns=['Unnamed: 0'], inplace=True)

## Splitting the Dataset into Training, Validation and Testing Data.

In [7]:
covid_df = pd.concat([x,y],axis=1)
covid_df

,cough,fever,sore_throat,shortness_of_breath,head_ache,Female,Male,Age above 60,Abroad,Contact with confirmed,Result
0,0,0,0,0,0,1,0,0,0,0,0
1,1,0,0,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,1,0,0,0,0
3,1,0,0,0,0,1,0,0,0,0,0
4,1,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
520011,1,0,0,0,0,0,1,0,0,1,1
520012,1,0,0,0,1,1,0,0,0,0,1
520013,0,0,0,0,0,0,1,0,0,0,1
520014,0,1,0,0,0,0,1,0,0,0,1


In [8]:
train_df,valid_df = train_test_split(covid_df,test_size=0.20,random_state=1)
train_df,test_df = train_test_split(train_df,test_size=0.10,random_state=1)

In [9]:
print(train_df.shape)
print(valid_df.shape)
print(test_df.shape)

(374410, 11)
(104004, 11)
(41602, 11)


In [10]:
train_df = train_df.reset_index()
valid_df = valid_df.reset_index()
test_df = test_df.reset_index()

In [11]:
train_df.drop(columns=['index'],inplace=True)
valid_df.drop(columns=['index'],inplace=True)
test_df.drop(columns=['index'],inplace=True)

In [12]:
independent_cols = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache',
                    'Female', 'Male', 'Age above 60', 'Abroad', 'Contact with confirmed']

target_col = ['Result']

In [13]:
x_train = train_df[independent_cols].copy()
y_train = train_df[target_col].copy()

x_valid = valid_df[independent_cols].copy()
y_valid = valid_df[target_col].copy()

x_test = test_df[independent_cols].copy()
y_test = test_df[target_col].copy()

## HyperParameter Tuning for RandomForest Classifier
The main parameters used by a Random Forest Classifier are:
- criterion = the function used to evaluate the quality of a split.
- max_depth = maximum number of levels allowed in each tree.
- max_features = maximum number of features considered when splitting a node.
- min_samples_leaf = minimum number of samples which can be stored in a tree leaf.
- min_samples_split = minimum number of samples necessary in a node to cause node splitting.
- n_estimators = number of trees in the ensemble.

### 1. HyperParameter Tuning - Randomized Search CV.

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [ ]:
rf=RandomForestClassifier()
skfold=StratifiedKFold(n_splits=3)
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=30,cv=skfold, verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(x_train,y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed: 65.8min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 130.3min finished
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
                   error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_sp...
                   pa

In [ ]:
rf_randomcv.best_params_

{'criterion': 'gini',
 'max_depth': 340,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 14,
 'n_estimators': 2000}

In [ ]:
best_random_grid=rf_randomcv.best_estimator_

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
y_pred=best_random_grid.predict(x_valid)
print(confusion_matrix(y_valid,y_pred))
print("Accuracy Score {}".format(accuracy_score(y_valid,y_pred)))
print("Classification report: {}".format(classification_report(y_valid,y_pred)))

[[47345  4718]
 [10581 41360]]
Accuracy Score 0.8528998884658282
Classification report:               precision    recall  f1-score   support

           0       0.82      0.91      0.86     52063
           1       0.90      0.80      0.84     51941

    accuracy                           0.85    104004
   macro avg       0.86      0.85      0.85    104004
weighted avg       0.86      0.85      0.85    104004



In [ ]:
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials

In [ ]:
space = {'criterion': hp.choice('criterion', ['entropy', 'gini']),
        'max_depth': hp.quniform('max_depth', 10, 1200, 10),
        'max_features': hp.choice('max_features', ['auto', 'sqrt','log2', None]),
        'min_samples_leaf': hp.uniform('min_samples_leaf', 0, 0.5),
        'min_samples_split' : hp.uniform ('min_samples_split', 0, 1),
        'n_estimators' : hp.choice('n_estimators', [10, 50, 300, 750, 1200,1300,1500])
    }

In [ ]:
def objective(space):
    model = RandomForestClassifier(criterion = space['criterion'], max_depth = space['max_depth'],
                                 max_features = space['max_features'],
                                 min_samples_leaf = space['min_samples_leaf'],
                                 min_samples_split = space['min_samples_split'],
                                 n_estimators = space['n_estimators'], 
                                 )
    
    accuracy = cross_val_score(model, x_train, y_train, cv = skfold).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return {'loss': -accuracy, 'status': STATUS_OK }

In [ ]:
trials = Trials()
best = fmin(fn= objective,
            space= space,
            algo= tpe.suggest,
            max_evals = 80,
            trials= trials)
best

  0%|          | 0/80 [00:00<?, ?it/s, best loss: ?]

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)



  1%|▏         | 1/80 [03:34<4:42:27, 214.52s/it, best loss: -0.5300552838608027]

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)



  2%|▎         | 2/80 [05:19<3:14:55, 149.94s/it, best loss: -0.6564301153067627]

...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)



 99%|█████████▉| 79/80 [2:34:54<02:05, 125.90s/it, best loss: -0.838431132087098]

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)



100%|██████████| 80/80 [2:39:13<00:00, 119.42s/it, best loss: -0.838431132087098]


{'criterion': 1,
 'max_depth': 1030.0,
 'max_features': 3,
 'min_samples_leaf': 0.0007591618968705617,
 'min_samples_split': 0.21837298391834725,
 'n_estimators': 2}

In [ ]:
crit = {0: 'entropy', 1: 'gini'}
feat = {0: 'auto', 1: 'sqrt', 2: 'log2', 3: None}
est = {0: 10, 1: 50, 2: 300, 3: 750, 4: 1200,5:1300,6:1500}


print(crit[best['criterion']])
print(feat[best['max_features']])
print(est[best['n_estimators']])

gini
None
300


In [ ]:
best['min_samples_leaf']

0.0007591618968705617

In [ ]:
trainedforest = RandomForestClassifier(criterion = crit[best['criterion']], max_depth = best['max_depth'], 
                                       max_features = feat[best['max_features']], 
                                       min_samples_leaf = best['min_samples_leaf'], 
                                       min_samples_split = best['min_samples_split'], 
                                       n_estimators = est[best['n_estimators']]).fit(x_train,y_train)
predictionforest = trainedforest.predict(x_valid)
print(confusion_matrix(y_valid,predictionforest))
print(accuracy_score(y_valid,predictionforest))
print(classification_report(y_valid,predictionforest))
acc5 = accuracy_score(y_valid,predictionforest)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[[43223  8840]
 [ 8166 43775]]
0.8364870581900696
              precision    recall  f1-score   support

           0       0.84      0.83      0.84     52063
           1       0.83      0.84      0.84     51941

    accuracy                           0.84    104004
   macro avg       0.84      0.84      0.84    104004
weighted avg       0.84      0.84      0.84    104004



## 3. HyperParameter Tuning - Genetic Algorithms

In [15]:
!pip install TPOT

     |████████████████████████████████| 87 kB 6.4 MB/s 
     |████████████████████████████████| 160 kB 47.0 MB/s 
     |████████████████████████████████| 173.5 MB 43 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11952 sha256=9657c3115cc028182a33b982a24dcce7adbf09e9224f2ab8adcf0e6aeadae2a5
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [14]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
param = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(param)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [ ]:
tpot_classifier = TPOTClassifier(generations= 5, population_size= 24, offspring_size= 12,
                                 verbosity= 2, early_stop= 12,
                                 config_dict={'sklearn.ensemble.RandomForestClassifier': param}, 
                                 cv = 3, scoring = 'accuracy')
tpot_classifier.fit(x_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Optimization Progress:   0%|          | 0/84 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8536898044783531

Generation 2 - Current best internal CV score: 0.8536898044783531

Generation 3 - Current best internal CV score: 0.8536898044783531

Generation 4 - Current best internal CV score: 0.8536898044783531


TPOT closed during evaluation in one generation.


TPOT closed prematurely. Will use the current best pipeline.
